In [0]:


date='2020-07-01'

def poc(date):
    new_app_est_df = spark.read.option("basePath", "s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/").parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date="+date)
    new_app_est_df.createOrReplaceTempView("new_app_est")
    app_est_category_rank_df = spark.read.option("basePath", "s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-category-rank.v1/fact").parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-category-rank.v1/fact/granularity=daily/date="+date)
    date = "'"+date+"'"
    app_est_category_rank_df.createOrReplaceTempView("app_est_category_rank")
    app_est_df = spark.sql("""
        select new_app_est.app_id,
               new_app_est.country_code,
               new_app_est.free_app_download,
               new_app_est.paid_app_download,
               new_app_est.revenue,
               new_app_est.revenue_iap,
               new_app_est.revenue_non_iap,
               new_app_est.granularity,
               new_app_est.date,
               new_app_est.device_code,
               category_id
        from app_est_category_rank
        left join new_app_est
        on new_app_est.app_id = app_est_category_rank.app_id and 
        new_app_est.country_code = app_est_category_rank.country_code and
        new_app_est.granularity = app_est_category_rank.granularity and 
        new_app_est.date =  app_est_category_rank.date and 
        new_app_est.device_code = app_est_category_rank.device_code 
               """)
    
    
    app_est_df.createOrReplaceTempView("app_est")
    
   
    dna_mapping_df = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-dna/unified/dna.mapping-log.v1/dimension").select("publisher_id", "app_id", "end_date","start_date").where("update_date ='2020-09-21'")
    dna_mapping_df.createOrReplaceTempView("dna_mapping")
    
    app_est_publisher_dna = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-publisher-dna-log.v1/fact/").where(" date = "+ date)
    app_est_publisher_dna.createOrReplaceTempView("app_est_publisher")
    
    app_est_category_publisher_df = spark.sql("""
        select free_app_download,
              paid_app_download,
              revenue,
              revenue_iap,
              revenue_non_iap,
              country_code,
              category_id,
              granularity,
              date,
              device_code, 
              publisher_id,
              app_est.app_id
        from app_est 
        left join dna_mapping on app_est.app_id= dna_mapping.app_id
        where date >=  start_date and (end_date is null or date <=  end_date)
    """)
    app_est_category_publisher_df.createOrReplaceTempView("app_est_category_publisher")
    
    
    
    
    
    app_sum_est_category_publisher_df = spark.sql("""
        select sum(free_app_download) as free_app_download,
              sum(paid_app_download) as paid_app_download,
              sum(revenue) as revenue,
              sum(revenue_iap) as revenue_iap,
              sum(revenue_non_iap) as revenue_non_iap,
              country_code,
              category_id,
              granularity,
              date,
              device_code, 
              publisher_id
        from app_est_category_publisher 
        group by date,granularity,device_code,category_id,country_code,publisher_id
    """)
    app_sum_est_category_publisher_df.createOrReplaceTempView("app_sum_est_category_publisher_df")
    
    
    app_sum_est_category_publisher_df.cache()
    app_est_publisher_df = spark.sql("""
        select est_free_app_download as free_app_download, 
               est_paid_app_download as paid_app_download, 
               est_revenue as revenue,
               est_revenue_iap as revenue_iap,
               est_revenue_non_iap as revenue_non_iap,
               country_code,
               category_id,
               granularity,
               date,
               device_code,
               publisher_id 
        from app_est_publisher
        where granularity = 'daily'
    """)
    app_est_publisher_df.cache()
    app_est_publisher_df.createOrReplaceTempView("app_est_publisher_df")
    
    
    
    app_est_category_publisher_count = app_sum_est_category_publisher_df.count()
    app_est_category_publisher_diff = app_sum_est_category_publisher_df.exceptAll(app_est_publisher_df)
    app_est_category_publisher_diff_count = app_est_category_publisher_diff.count()
    
    app_est_publisher_count = app_est_publisher_df.count()
    app_est_publisher_diff = app_est_publisher_df.exceptAll(app_sum_est_category_publisher_df)
    app_est_publisher_diff_count = app_est_publisher_diff.count()
    print(date)
    print("app_est_category_publisher_count:",app_est_category_publisher_count)
    print("app_est_category_publisher_diff_count:",app_est_category_publisher_diff_count)
    app_different_rate = 100*app_est_category_publisher_diff_count/app_est_category_publisher_count
    print("app_different_rate:"+str(app_different_rate)+'%')
    
    print("app_est_publisher_count:",app_est_publisher_count)
    print("app_est_publisher_diff_count:",app_est_publisher_diff_count)
    publisher_different_rate = 100*app_est_publisher_diff_count/app_est_publisher_count
    print("publisher_different_rate:"+str(publisher_different_rate)+'%')
    
    
    print("------------------------------------------------------")
    print("Top 1000 pyblisher order by free_app_download")
    app_top_free_download_df = spark.sql("""
        select * from app_sum_est_category_publisher_df order by free_app_download desc,publisher_id  limit 1000 
    """)
    publisher_top_free_download_df = spark.sql("""
        select * from app_est_publisher_df order by free_app_download desc,publisher_id limit 1000
    """)
    
    app_top_free_download_diff_rate = 100*app_top_free_download_df.exceptAll(publisher_top_free_download_df).count()/1000
    print("app_top_free_download_diff_rate:"+str(app_top_free_download_diff_rate)+"%")
    
    publisher_top_free_download_rate = 100*publisher_top_free_download_df.exceptAll(app_top_free_download_df).count()/1000
    print("publisher_top_free_download_rate:"+str(publisher_top_free_download_rate)+"%")
    
    
    print("------------------------------------------------------")
    
    print("Top 1000 pyblisher order by paid_app_download")
    app_top_paid_download_df = spark.sql("""
        select * from app_sum_est_category_publisher_df order by paid_app_download desc,publisher_id limit 1000
    """)
    publisher_top_paid_download_df = spark.sql("""
        select * from app_est_publisher_df order by paid_app_download desc,publisher_id limit 1000
    """)
    
    app_top_paid_download_diff_rate = 100*app_top_paid_download_df.exceptAll(publisher_top_paid_download_df).count()/1000
    print("app_top_paid_download_diff_rate:"+str(app_top_paid_download_diff_rate)+"%")
    
    publisher_top_paid_download_rate = 100*publisher_top_paid_download_df.exceptAll(app_top_paid_download_df).count()/1000
    print("publisher_top_paid_download_rate:"+str(publisher_top_paid_download_rate)+"%")
    print("------------------------------------------------------")
    
    
    print("Top 1000 pyblisher order by revenue")
    app_top_revenue_df = spark.sql("""
        select * from app_sum_est_category_publisher_df order by revenue desc,publisher_id limit 1000
    """)
    publisher_top_revenue_df = spark.sql("""
        select * from app_est_publisher_df order by revenue desc,publisher_id limit 1000
    """)
    
    app_top_revenue_diff_rate = 100*app_top_revenue_df.exceptAll(publisher_top_revenue_df).count()/1000
    print("app_top_revenue_diff_rate:"+str(app_top_revenue_diff_rate)+"%")
    
    publisher_top_revenue_rate = 100*publisher_top_revenue_df.exceptAll(app_top_revenue_df).count()/1000
    print("publisher_top_revenue_rate:"+str(publisher_top_revenue_rate)+"%")
    print("=========================================================")
    app_est_publisher_df.unpersist()
    app_sum_est_category_publisher_df.unpersist()
    
dates = ["2017-07-01", "2017-07-02", "2017-07-03", "2017-07-04", "2017-07-05", "2017-07-06", "2017-07-07", "2017-07-08", "2017-07-09", "2017-07-10",
 "2017-07-11", "2017-07-12", "2017-07-13", "2017-07-14", "2017-07-15", "2017-07-16", "2017-07-17", "2017-07-18", "2017-07-19", "2017-07-20",
 "2017-07-21", "2017-07-22", "2017-07-23", "2017-07-24", "2017-07-25", "2017-07-26", "2017-07-27", "2017-07-28", "2017-07-29", "2017-07-30",
 "2017-07-31"
        ]
for d in dates:
    poc(d)
